In [ ]:
#這是jupyter notebook的magic word˙
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from IPython import display

# 視覺親緣鑑定

In [ ]:
import os
#判斷是否在jupyter notebook上
def is_in_ipython():
    "Is the code running in the ipython environment (jupyter including)"
    program_name = os.path.basename(os.getenv('_', ''))

    if ('jupyter-notebook' in program_name or # jupyter-notebook
        'ipython'          in program_name or # ipython
        'jupyter' in program_name or  # jupyter
        'JPY_PARENT_PID'   in os.environ):    # ipython-notebook
        return True
    else:
        return False


#判斷是否在colab上
def is_in_colab():
    if not is_in_ipython(): return False
    try:
        from google import colab
        return True
    except: return False

#判斷是否在kaggke_kernal上
def is_in_kaggle_kernal():
    if 'kaggle' in os.environ['PYTHONPATH']:
        return True
    else:
        return False

if is_in_colab():
    from google.colab import drive
    drive.mount('/content/gdrive')

In [ ]:
os.environ['TRIDENT_BACKEND'] = 'pytorch'

if is_in_kaggle_kernal():
    os.environ['TRIDENT_HOME'] = './trident'
    
elif is_in_colab():
    os.environ['TRIDENT_HOME'] = '/content/gdrive/My Drive/trident'

#為確保安裝最新版 

!pip uninstall tridentx -y
!pip install ../input/trident/tridentx-0.7.3.20-py3-none-any.whl --upgrade
#!pip install cupy

import json
import copy
import numpy as np
#調用trident api
import trident as T
from trident import *

from trident.layers.pytorch_initializers import orthogonal
import random
from tqdm import tqdm
import glob
import scipy
import time

In [ ]:
make_dir_if_need('./train-faces')
!unzip ../input/recognizing-faces-in-the-wild/train-faces.zip -d ./train-faces

In [ ]:
family_labels=glob.glob('./train-faces/F*')
family_labels=list(sorted(set([f.replace('./train-faces/','') for f in family_labels])))
print(len(family_labels))
print(family_labels[:10])

person_labels=glob.glob('./train-faces/F*/MID*')
person_labels=list(sorted(set([f.replace('./train-faces/','') for f in person_labels])))
print(len(person_labels))
print(person_labels[:10])

In [ ]:
images=[]
families=[]
persons=[]
train_faces=glob.glob('./train-faces/F*/MID*/*.jpg')
print(len(train_faces))
for i in tqdm(range(len(train_faces))):
    im_path=train_faces[i]
    cols=im_path.split('/')
    images.append(im_path)
    families.append(family_labels.index(cols[-3]))
    persons.append(person_labels.index(cols[-3]+'/'+cols[-2]))

print(images[:10])
print(families[:10])
print(persons[:10])
    

In [ ]:
ds1=ImageDataset(images,object_type=ObjectType.rgb, symbol='images')
ds2=LabelDataset(families,object_type=ObjectType.classification_label, symbol='families_label')
ds3=LabelDataset(persons, object_type=ObjectType.classification_label,symbol='persons_label')


zipdata=ZipDataset(ds2,ds3)
data_provider=DataProvider(traindata=Iterator(data=ds1,label=zipdata))
data_provider.image_transform_funcs=[
    Resize(output_size=(112,112)),
    RandomTransform(rotation_range= 15, zoom_range= 0.02, shift_range= 0.00,shear_range = 0.02,random_flip= 0.15),
    RandomAdjustGamma(gamma_range=(0.6,1.4)),
    RandomAdjustContrast(value_range=(0.6,1.4)),
    RandomAdjustHue(scale=(-0.3,0.3)),#調整色相
    RandomAdjustSaturation(scale=(0.6,1.4)),#調整飽和度
    RandomBlur(scale=(3,7)),#隨機模糊
    SaltPepperNoise(prob=0.001),#椒鹽噪音
    RandomErasing(size_range=(0.08,0.2),transparency_range=(0.4,0.6),transparancy_ratio=0.8),
    Normalize(127.5,127.5)]


In [ ]:
img_data,label1,label2=data_provider.next()
print(img_data.shape)
print(label1.shape)
print(label2.shape)

In [ ]:
data_provider.preview_images()

In [ ]:
from trident.models import arcfacenet
num_faces=10575
#標準生成結構
#不包含原有分類器
se_resnet50 =arcfacenet.SEResNet_IR_50_512(include_top=False,
             pretrained=True,
             freeze_features=True,
             input_shape=(3,112,112))

#加入output_layer
se_resnet50.model.add_module('output_layer', 
    Sequential(
        Dropout(dropout_rate=0.4),
        Flatten(),
        Dense((512),use_bias=False,keep_output=True),
    ))

#se_resnet50.model.output_layer[0].inplace = False
#從之前預訓練的人臉識別arcFace還原權重
se_resnet50.model.add_module('l2norm',L2Norm())
se_resnet50.model.add_module('fc',Dense((num_faces),use_bias=False,weights_norm='l2'))
if os.path.exists('../input/face-recognition/Models/arcface.pth.tar'):
    se_resnet50.load_model('../input/face-recognition/Models/arcface.pth.tar')

se_resnet50.summary()

所以接下來我們要稍微改裝一下我們預訓練好的arcface，首先我們的輸出必須改成兩組，一個是親屬關係，另一個是個人，等於是我們的人臉識別模型同時要識別這兩者差異。

In [ ]:
se_resnet50.model.trainable=False
se_resnet50.model.body[23].trainable=True
se_resnet50.model.remove_at(-1)
results=ModuleDict({'family':Dense(786),'person':Dense(3965),'features':Identity()},is_multicasting=True)
se_resnet50.model.add_module('results',results)


se_resnet50.summary()

在損失函數上仍然是使用arcFace，只不過這邊的分類要變成兩個階層，一個是判斷是哪一個家族，再來才是判斷是哪一個個人。由於兩者層級不同，根據組內差距小、組間差距大的原則，可以得知家族間的差異應該要大於個人間的差異。因此我們把同樣的arcFace的損失函數複製兩份，修改兩者的Forward部分，同時我們把家族的損失函數的Margin以及Scale都放大一倍。  
同時也在此定義了家族與個人分類的準確率函數。

In [ ]:

class Family_ArcMarginProductLoss(Layer):
    def __init__(self, scale=64.0, margin=1, easy_margin=False, name='ArcMarginProductLoss'):
        super(Family_ArcMarginProductLoss, self).__init__()
        self._name=name
        self.scale = scale
        self.m = margin
        self.easy_margin = easy_margin
        self.cos_m = math.cos(margin)
        self.sin_m = math.sin(margin)

        # make the function cos(theta+m) monotonic decreasing while theta in [0°,180°]
        self.th = math.cos(math.pi - margin)
        self.mm = math.sin(math.pi - margin) * margin
    
        self.base_loss=CrossEntropyLoss(reduction='mean')


    def forward(self, family, families_label,**kwargs):
        # cos(theta)
        try:
            cosine=family
            target=families_label
            # cos(theta + m)
            sine = sqrt(1.0 - pow(cosine, 2))
            phi = cosine * self.cos_m - sine * self.sin_m

            if self.easy_margin:
                phi = where(cosine > 0, phi, cosine)
            else:
                phi = where((cosine - self.th) > 0, phi, cosine - self.mm)

            one_hot = zeros_like(cosine,requires_grad=True)
            one_hot.scatter(1, target.view(-1, 1), 1)

            output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
            output = output * self.scale
        except Exception as e:
            print(e)
            PrintException()

        loss = self.base_loss(output, target)
        return loss
    
    
class Person_ArcMarginProductLoss(Layer):
    def __init__(self, scale=32.0, margin=0.50, easy_margin=False, name='ArcMarginProductLoss'):
        super(Person_ArcMarginProductLoss, self).__init__()
        self._name=name
        self.scale = scale
        self.m = margin
        self.easy_margin = easy_margin
        self.cos_m = math.cos(margin)
        self.sin_m = math.sin(margin)

        # make the function cos(theta+m) monotonic decreasing while theta in [0°,180°]
        self.th = math.cos(math.pi - margin)
        self.mm = math.sin(math.pi - margin) * margin
    
        self.base_loss=CrossEntropyLoss(reduction='mean')


    def forward(self, person, persons_label,**kwargs):
        # cos(theta)
        try:
            cosine=person
            target=persons_label
            # cos(theta + m)
            sine = sqrt(1.0 - pow(cosine, 2))
            phi = cosine * self.cos_m - sine * self.sin_m

            if self.easy_margin:
                phi = where(cosine > 0, phi, cosine)
            else:
                phi = where((cosine - self.th) > 0, phi, cosine - self.mm)

            one_hot = zeros_like(cosine,requires_grad=True)
            one_hot.scatter(1, target.view(-1, 1), 1)

            output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
            output = output * self.scale
        except Exception as e:
            print(e)
            PrintException()

        loss = self.base_loss(output, target)
        return loss
    
def center_loss(features,families_label):
    return CenterLoss(num_classes=786, feat_dim=512)(features,families_label)
    
def family_accuracy(family, families_label):
    return accuracy(family, families_label)

    
def person_accuracy(person, persons_label):
    return accuracy(person, persons_label)

在這邊我們仍然使用AdaBelief優化器，學習速率在1e-3，定且加入剛才定義好的損失函數與評估函數。

In [ ]:
se_resnet50.with_optimizer(optimizer=AdaBelief, lr=1e-3, betas=(0.9, 0.999)) \
    .with_loss(Family_ArcMarginProductLoss(scale=64.0, margin=1.00, easy_margin=False)) \
    .with_loss(Person_ArcMarginProductLoss(scale=32.0, margin=0.50, easy_margin=False)) \
    .with_loss(center_loss) \
    .with_metric(family_accuracy, name='family_accuracy') \
    .with_metric(person_accuracy, name='person_accuracy') \
    .with_regularizer('l2',reg_weight=1e-5) \
    .with_accumulate_grads(10)\
    .with_model_save_path('./Models/arcface_family.pth')\
    .with_automatic_mixed_precision_training()

In [ ]:
plan=TrainingPlan()\
    .add_training_item(se_resnet50, name='arcface')\
    .with_data_loader(data_provider)\
    .repeat_epochs(20)\
    .with_batch_size(64)\
    .print_progress_scheduling(10,unit='batch')\
    .out_sample_evaluation_scheduling(100,unit='batch')\
    .display_loss_metric_curve_scheduling(200,unit='batch',imshow=True)\
    .save_model_scheduling(50,unit='batch')

In [ ]:
plan.start_now()